In [25]:
import numpy as np
import pandas as pd
df = pd.read_csv('tennis.csv')
df.drop('day', 1, inplace=True)
df.head()

,outlook,temp,humidity,wind,play
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes


In [100]:
def entropies(data, col, label):
    ents = []
    props = data[label].value_counts()/len(data)
    ents.append(-sum(props * np.log2(props)))
    col = data[col]
    for val in col.unique():
        sub = data.loc[col == val]
        props = sub[label].value_counts()/len(sub)
        ents.append(-sum(props * np.log2(props)))
    return pd.Series(ents)

def gain(data, col, label):
    props = sum([[1], data[col].value_counts()/len(data)], [])
    props = pd.Series([1]).append(data[col].value_counts()/len(data))
    ents = entropies(data, col, label)
    ents.index = props.index
    product = props * ents
    return product.iloc[0] - sum(product.iloc[1:])

def find_split(data, label):
    scores = []
    for col in data.drop(label, 1).columns:
        scores.append(gain(data, col, label))
    return data.columns[scores.index(max(scores))]

def make_tree(data, label, val = None):
    tree = []
    if data.shape[1] > 1 and data[label].nunique() != 1:
        on = find_split(data, label)
        tree.append(on)
        for val in data[on].unique():
            split = make_tree(data.loc[data[on] == val].drop(on, 1), label)
            if split != []:
                tree.append(split)
    return tree

In [101]:
make_tree(df, 'play')

['outlook', ['humidity'], ['wind']]

In [102]:
new_data = pd.DataFrame(np.matrix([['a','s','d','f',1],
                                   ['f','f','f','d',0],
                                   ['d','d','f','d',1],
                                   ['a','a','f','s',0],
                                   ['s','f','f','s',1],
                                   ['d','a','f','a',0],
                                   ['f','f','s','d',1],
                                   ['d','a','s','f',0],
                                   ['s','f','s','a',1],
                                   ['s','s','a','a',0],
                                   ['s','f','d','a',1],
                                   ['f','d','a','a',0],
                                   ['a','d','d','a',1],
                                   ['d','d','f','s',0]]))

In [104]:
%time make_tree(new_data, 4)

CPU times: user 135 ms, sys: 3.66 ms, total: 139 ms
Wall time: 138 ms


[2, [0, [3]], [0]]